# Scoring Communication

TODO: EA will make this more comprehensive

This tutorial will guide users from the processed expression matrix to a "communication matrix" for each context. A communication matrix has as columns cell type pairs, as rows ligand-receptor pairs, and as value a communication score indicating the strenght of the interaction. 

This tutorial show how to do this with cell2cell's native functions. See tutorial 02B to replace these steps with external tools run through LIANA

In [98]:
import cell2cell as c2c

import scanpy as sc
import numpy as np
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

#from cell2cell....import CellComposition
seed = 888
np.random.seed(seed)

In [ ]:
out_path = '/data3/hratch/c2c_general/'

First, load the processed object and assigned the expression matrix. In our case, we will proceed with log(1+CPM) data rather than the batch-corrected data, since this is the input to most CCC tools and does not introduce negative counts.  

In [86]:
balf_corrected = sc.read_h5ad(out_path + 'batch_corrected_balf_covid.h5ad') 

## assign batch corrected data as expression matrix
# rnaseq = balf_corrected.to_df() 
# meta = balf_corrected.obs

# assign loc(1+CPM) data as expression matrix
log1p = balf_corrected.raw.to_adata()
rnaseq = log1p.to_df().T
meta = log1p.obs

Or if you used R with tutorial 1B:

In [ ]:
balf_corrected = sc.read_h5ad(out_path + 'Rbatch_corrected_balf_covid.h5ad') 

## assign batch corrected data as expression matrix
# rnaseq = balf_corrected.to_df()
# rnaseq.index = balfR.var.features

# assign loc(1+CPM) data as expression matrix
#TODO: couldnt find the log(1+CPM) assay in the saved file

# 1) Aggregate Expression

We split the expression matrix by sample and aggregate into cell types using the fraction of non-zero genes:

In [120]:
rnaseq_matrices = {}
for sample in tqdm(meta.Sample_ID.unique()):
    
    # split expression matrix by sample
    meta_context = meta[meta.Sample_ID == sample]
    cells = list(meta_context.index) 
    tmp_data = rnaseq.loc[cells,:]
    
    meta_context.index.name = 'barcode'
    # aggregate by fraction of non-zero genes
    rnaseq_matrices[sample] = c2c.preprocessing.aggregate_single_cells(rnaseq_data=tmp_data,
                                                  metadata=meta_context,
                                                  barcode_col='barcode',
                                                  celltype_col='cell_type',
                                                  method='nn_cell_fraction',
                                                 )

100%|███████████████████████████████████████████| 12/12 [01:05<00:00,  5.49s/it]


# 2) LR Pairs

In [121]:
lr_pairs = pd.read_csv('https://raw.githubusercontent.com/LewisLabUCSD/Ligand-Receptor-Pairs/master/Human/Human-2020-Jin-LR-pairs.csv')
int_columns = ('ligand_symbol', 'receptor_symbol')
lr_pairs = c2c.preprocessing.ppi.remove_ppi_bidirectionality(ppi_data=lr_pairs, 
                                                             interaction_columns=int_columns
                                                             )

ppi_functions = dict()

for idx, row in lr_pairs.iterrows():
    ppi_label = row[int_columns[0]] + '^' + row[int_columns[1]]
    ppi_functions[ppi_label] = row['annotation']

Removing bidirectionality of PPI network


# 3) Calculating a LR Communication Score

Here, we calculate a communication score from just the gene expression of the ligands and receptors. In Tutorial #3, we will further discuss how to account for cell composition.


With the internal methods, we can combine communication scoring with tensor building (in tutorial 2B we will see how to score each sample separately using LIANA and subsequently build the tensor):

TODO: implementation and discussion of (HB utility script docstrings from CodeOcean has first two points)
-outer vs inner for both LRs and cells
-filling with 0 vs NaN
-masking: tensor decomposition itself must have no NaNs. Masking will ignore any index in the mask. So, we mask NaN values only (not true zeros) and after generating the mask, fill NaNs with zeros. 

In [123]:
tensor = c2c.tensor.InteractionTensor(rnaseq_matrices=list(rnaseq_matrices.values()),
                                      ppi_data=lr_pairs,
                                      context_names=list(rnaseq_matrices.keys()),
                                      how='inner',
                                      complex_sep='&',
                                      interaction_columns=int_columns,
                                      communication_score='expression_mean',
                                     )

Getting expression values for protein complexes
Building tensor for the provided context


Save the tensor

In [125]:
c2c.io.export_variable_with_pickle(tensor, out_path + 'tensor_internal.pkl')

/data3/hratch/c2c_general/tensor_internal.pkl  was correctly saved.
